<a href="https://colab.research.google.com/github/athahibatullah/llm-from-scratch/blob/main/02%20-%20Working%20with%20Text%20Data/ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **2.1 Understanding Word Embeddings**

Deep neural networks including LLMs cannot process raw text directly since text is categorical. Categorical mean it isn't compatible with the mathematical operations used to implement and train neural networks. Therefore, we need a way to represent words as continuous-valued vectors. The concept of converting data into a vector format is often referred to as embedding. Different data formats require distinct embedding models, for example an embedding model designed for text would not be suitable for embedding audio or video data.
<img src="https://drive.google.com/uc?id=1Uu1CZJM8mGLImwADawl9CCB0xCicfocF">

Embedding is a mapping from discrete objects, such as words, images, or even entire documents, to points in a continuous vector space. The primary purpose of embeddings is to convert nonnumeric data into a format that neural networks can process. Text embedding is not limited to word embeddings, there are also embeddings for sentence, paragraphs, or whole documents. Sentence or paragraph embeddings are popular choices for Retrieval-Augmented Generation (RAG). Retrieval-Augmented Generation combines generation (like producing text) with retrieval (like searching an external knowledge base) to pull relevant information when generating text, RAG isn't included in this scope of building LLM from scratch. We will focus on word embeddings.

One of the earlier and most popular examples of word embeddings algorithm is Word2Vec. Word2Vec trained neural network architecture to generate word embeddings by predicting the context of a word given the target word or vice versa. The main idea of Word2Vec is word that appear in similar contexts tend to have similar meanings. This mean if we project them into two-dimensional word embeddings for visualization purpose, similar terms are clustered together. Word embeddings can have varying dimensions, from one to thousands. More dimension is better but with more computational cost.

<img src="https://drive.google.com/uc?id=16PcjL0A-UuN9O3CxZ4cjgGh2DtjEZ_nl">

While we can use pretrained models such as Word2Vec to generate embeddings for machine learning models, LLMs commonly produce their own embeddings that are part of the input layer and are updated during training. The advantage of optimizing the embeddings as part of the LLM training instead of using Word2Vec is that the embeddings are optimized to the specific task and data at hand.

High-dimensional embeddings present a challenge for visualization because humans are limited to seeing a 3 or fewer dimensions. When working with LLMs, we typically use embedding with a much higher dimensionality. The smallest GPT-2 models (117M and 125M paramters) use an embedding size of 768 dimensions and the largest GPT-3 model (175B parameters) use an embedding size of 12288 dimensions.

# **2.2 Tokenizing Text**

Let's split input text into individual tokens first as it's required preprocessing step for creating embeddings for an LLM.
<img src="https://drive.google.com/uc?id=1rkJu1MKBaQbVSEETK07rxCulowe1COZQ">

The text we will tokenize for LLM training is "The Verdict", a short story by Edith Wharton. The text is available on Wikisource: https://en.wikisource.org/wiki/The_Verdict.

First, we will load the text file: the-verdict.txt, count total number of character and print the first 100 character of the file.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import urllib.request

if not os.path.exists("the-verdict.txt"):
    url = ("https://raw.githubusercontent.com/rasbt/"
           "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
           "the-verdict.txt")
    file_path = "the-verdict.txt"
    urllib.request.urlretrieve(url, file_path)

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


We can start splitting the text by using a short sentence first to make sure our splitting function works as intended.

In [ ]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [ ]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [ ]:
# Strip whitespace from each item and then filter out any empty strings.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [ ]:
# Handle punctuations
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


After successfully create splitting function, we can apply it first to the previous first 100 characters and after that apply it to the entire document.

Note that we refrain from making all text lower-case because capitalization helps LLMs distinguish between proper nouns and common nouns, understand sentence structure, and learn to generate text with proper capitalization.

When developing a simple tokenizer, it's up to us to decide whether we should encode whitespaces as separate characters or just remove them as it's depends on our application and its requirements. Removing whitespaces reduces the memory and computing requirements, but can also be helpful if we train models that are sensitive to the exact structure of the text like for example python code as it's sensitive to indentation and spacing. In this example, we remove whitespaces for simplicity and brevity of the tokenized outputs. Later, we will switch to a tokenization scheme that includes whitespaces.

In [ ]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [ ]:
print(len(preprocessed))

4690


# **2.3 Converting Tokens into Token IDs**

After successfully extracting token from the text, we convert these tokens to an integer representation to produce the token IDs. To map the previously generated tokens into token IDs, we have to build a vocabulary first. This vocabulary defines how we map each unique word and special character to a unique integer.
<img src="https://drive.google.com/uc?id=1DgFbB-9X8qR9s_M13UUyuIhh8kyB2yPg">

Unique tokens mean duplicated word/punctuation will be counted as 1.

In [ ]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [ ]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [ ]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


set() function will return a list of unique token, and sorted() will sort the list. We will only print the first 50 vocabulary.

Next, our goal is to apply this vocabulary to convert our text into token IDs.

<img src="https://drive.google.com/uc?id=15xDpA-gvfAAghdZA06hLV6wGeLnG0gKr">


In [ ]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab # Stores the vocabulary as a class attribute for access in the encode and decode methods
        self.int_to_str = {i:s for s,i in vocab.items()} # Creates an inverse vocabulary that maps
                                                         # token into IDs back to the original text tokens

    def encode(self, text): # Processes input text into token IDs
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids): # Converts token IDs back into text
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

We create an inverse vocabulary that maps token IDs back to the original text tokens, an encode to process input text into token IDs, and decode to converts token IDs back into text.

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


We try our encode function and it's working. Next let's decode to see our text back.

In [ ]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

It successfully decoded our text into a readable human text.

# **2.4 Adding Special Context Tokens**

We need to modify the tokenizer to handle unknown words. We also need to address the usage and addition of special context tokens that can enhance a model's understanding of context or other relevant information in the text. These special tokens can include markers for unknown words and document boundaries. For example, we will modify the vocabulary and add special token to handle unknown words and end of text. End of text is needed to separate different dataset since LLM need to be trained using multiple books, articles, documents, or even separating different context in the same documents and this is where this special token come into play. For unknown words (words doesn't exist in current vocabulary), we name <|unk|>. For end of text, we name <|endoftext|>.

<img src="https://drive.google.com/uc?id=1QVadUQfaqZQm79uk32iS1o4wLT_T6zO6" >

<img src="https://drive.google.com/uc?id=1RHETuKRD5Zr89fH7nHP3lgS-QY77SrYG" >

To better understand, lets encode a new text with word that doesn't exist in current vocabulary

In [ ]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

It will return error since "Hello" token isn't registered in vocabulary. Now let's modify the code and add <|unk|> and <|endoftext|>.

In [ ]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [ ]:
len(vocab.items())

1132

In [ ]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


Now we successfully added our new token. After adding new token, we should modify our encode & decode function to use this new token.



In [ ]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [ # Replaces unknown words by <|unk|>
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text) # Replaces spaces before the specified
        return text

Now let's test our modified function.

In [ ]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [ ]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [ ]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

It's now added our new special tokens.

Depending on the LLM, some researchers also consider additional special tokens such as the following:

[BOS] (beginning of sequence): This token marks the start of a text. It signifies to the LLM where a piece of content begins.

[EOS] (end of sequence): This token is positioned at the end of a text and is especially useful when concatenating multiple unrelated texts, similar to <|endoftext|>. For instance, when combining two different Wikipedia articles or books, the [EOS] indicates where one ends and the next begins.

[PAD] (padding): When training LLMs with batch sizes larger than one, the batch might contain texts of varying lengths. To ensure all texts have the same length, the shorter texts are extended or "padded" using the [PAD] token, up to the length of the longest text in the batch.
The tokenizer used for GPT models does not need any of these tokens. <|endoftext|> is enough for simplicity. <|endoftext|> is analogous to the [EOS] and can also be used for padding. On later chapter when training on batched inputs introduced, we typically use a mask, meaning we don't attend to padded tokens. Morover, the tokenized used for GPT models also doesn't use <|unk|> for unknown words but instead use a byte pair encoding tokenizer, which breaks words down into subword units, which we will discuss next.

# **2.5 Byte Pair Encoding**

Byte Pair Encoding (BPE) was used to train LLM like GPT-2, GPT-3, and the original model used in ChatGPT. We will use library called tiktoken, which implements the BPE algorithm very efficiently based on source code in Rust.

Let's install the tiktoken first.

In [ ]:
pip install tiktoken

In [ ]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [ ]:
tokenizer = tiktoken.get_encoding("gpt2")

Once installed, let's do our encode and decode like before but this time with tiktoken.

In [ ]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [ ]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


We can make two noteworthy observations based on the token IDs and decoded text. First, the <|endoftext|> token is assigned a relatively large token ID, namely, 50256. In fact, the BPE tokenizer, which was used to train models such as GPT-2, GPT-3, and the original model used in ChatGPT, has a total vocabulary size of 50257, with <|endoftext|> being assigned the largest token ID.

Second, the BPE tokenizer encodes and decodes unknown words, such as someunknownPlace, correctly. The BPE tokenizer can handle any unknown word without using <|unk|> token. BPE algorithm breakdown words that aren't predefined in the vocabulary into smaller sub word unit or even down to characters, enabling it to handle out-of-vocabulary words.

<img src="https://drive.google.com/uc?id=1e4ejtAvUhcvMAgzn0tlFXjcVGZ0wvMWs">

The ability to break down unknown words into individual characters ensures that the tokenizer and, consequently, the LLM that is trained with it can process any text, even if it contains words that were not present in its training data.

BPE builds its vocabulary by iteratively merging frequent characters into sub words and frequent sub words into words. For example, BPE starts with adding all individual single characters to its vocabulary ("a", "b", etc.). Next, it merges character combinations that frequently occur together into sub words. For example, "d" and "e" may be merged into the subword "de," which is a common word in English like "define", "decode", "depend", "made", and "hidden".

## **Exercise**

In [ ]:
text = "Akwirw ier"

integers = tokenizer.encode(text)

print(integers)

[33901, 86, 343, 86, 220, 959]


In [ ]:
strings = tokenizer.decode(integers)
print(strings)

Akwirw ier


# **2.6 Data Sampling with a Sliding Window**

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
enc_sample = enc_text[50:]

In [ ]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [ ]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [ ]:
import torch
print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


In [ ]:
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [ ]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [ ]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [ ]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


In [ ]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
